In [ ]:
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import os

img = Image.open('panda.jpg')
w, h = img.size
print(f"Размер изображения: {w}x{h} пикселей")

# Отображение исходного изображения
plt.figure(figsize=(10, 8))
plt.imshow(img)
plt.title('Исходное изображение')
plt.axis('off')
plt.show()
# Преобразование изображения в numpy массив
x = np.array(img, dtype=np.float64)
print(f"Тип данных массива: {x.dtype}")
print(f"Форма массива: {x.shape}")
print(f"Количество каналов: {3 if len(x.shape) == 3 else 1}")

# Расчет размера исходного изображения в байтах
original_size_bytes = os.path.getsize(r'D:\1Programmirovanie\LLM универ\2 лаба\panda.jpg')
print(f"Размер исходного файла: {original_size_bytes} байт")

def compress_image_svd(image_array, r):
    """
    Сжимает изображение с использованием SVD разложения

    """
    # Проверяем, является ли изображение цветным или черно-белым
    if len(image_array.shape) == 3:
        # Цветное изображение (3 канала)
        compressed = np.zeros_like(image_array)

        # Применяем SVD к каждому каналу отдельно
        for channel in range(3):
            U, s, Vt = np.linalg.svd(image_array[:, :, channel], full_matrices=False)

            # Оставляем только r сингулярных значений
            s_truncated = np.diag(s[:r])
            U_truncated = U[:, :r]
            Vt_truncated = Vt[:r, :]

            # Восстанавливаем матрицу канала
            compressed[:, :, channel] = np.dot(U_truncated, np.dot(s_truncated, Vt_truncated))

        # Расчет коэффициента сжатия для цветного изображения
        original_size = image_array.shape[0] * image_array.shape[1] * 3
        compressed_size = r * (image_array.shape[0] + image_array.shape[1] + 1) * 3
    else:
        # Черно-белое изображение
        U, s, Vt = np.linalg.svd(image_array, full_matrices=False)

        # Оставляем только r сингулярных значений
        s_truncated = np.diag(s[:r])
        U_truncated = U[:, :r]
        Vt_truncated = Vt[:r, :]

        # Восстанавливаем матрицу
        compressed = np.dot(U_truncated, np.dot(s_truncated, Vt_truncated))

        # Расчет коэффициента сжатия для черно-белого изображения
        original_size = image_array.shape[0] * image_array.shape[1]
        compressed_size = r * (image_array.shape[0] + image_array.shape[1] + 1)

    # Ограничиваем значения пикселей
    compressed = np.clip(compressed, 0, 255)

    # Рассчитываем коэффициент сжатия
    compression_ratio = compressed_size / original_size

    return compressed.astype(np.uint8), compression_ratio

# Определяем количество сингулярных значений для тестирования
r_values = [1, 2, 10, 30, 100]
compressed_images = []
compression_ratios = []

# Сжимаем изображение для каждого значения r
for r in r_values:
    compressed_img, ratio = compress_image_svd(x, r)
    compressed_images.append(compressed_img)
    compression_ratios.append(ratio)
    print(f"r = {r}: коэффициент сжатия = {ratio:.4f}")

# Отображение результатов
plt.figure(figsize=(15, 10))

# Исходное изображение
plt.subplot(2, 3, 1)
plt.imshow(img)
plt.title(f'Исходное изображение\n{original_size_bytes:,} байт')
plt.axis('off')

# Сжатые изображения
for i, r in enumerate(r_values):
    plt.subplot(2, 3, i+2)
    plt.imshow(compressed_images[i])
    original_pixels = x.shape[0] * x.shape[1] * 3
    compressed_size_approx = int(original_pixels * compression_ratios[i])
    plt.title(f'r = {r}\n~{compressed_size_approx:,} байт\nсжатие: {compression_ratios[i]:.3%}')
    plt.axis('off')

plt.tight_layout()
plt.show()

# Анализ сингулярных значений для каждого канала
plt.figure(figsize=(12, 8))

for channel in range(3):
    U, s, Vt = np.linalg.svd(x[:, :, channel], full_matrices=False)

    plt.subplot(2, 2, channel+1)
    plt.plot(s[:100], 'b-', linewidth=2)
    plt.title(f'Сингулярные значения (канал {channel})')
    plt.xlabel('Индекс')
    plt.ylabel('Значение')
    plt.grid(True)

    # Отмечаем выбранные значения r
    for r in r_values:
        if r < 100:
            plt.axvline(x=r-1, color='r', linestyle='--', alpha=0.5)
            plt.text(r-1, s[r-1], f' r={r}', fontsize=10, verticalalignment='bottom')

plt.subplot(2, 2, 4)
# Суммарная энергия сингулярных значений
for channel in range(3):
    U, s, Vt = np.linalg.svd(x[:, :, channel], full_matrices=False)
    cumulative_energy = np.cumsum(s) / np.sum(s)
    plt.plot(cumulative_energy[:100], label=f'Канал {channel}')

plt.title('Накопленная энергия сингулярных значений')
plt.xlabel('Количество сингулярных значений')
plt.ylabel('Доля энергии')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Сравнение исходного и сильно сжатого изображения (r=10)
r = 10
compressed_img, ratio = compress_image_svd(x, r)

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(img)
plt.title('Исходное изображение')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(compressed_img)
plt.title(f'Сжатое изображение (r={r})')
plt.axis('off')

plt.subplot(1, 3, 3)
# Разница между изображениями
difference = np.abs(x.astype(float) - compressed_img.astype(float))
plt.imshow(difference.mean(axis=2).astype(np.uint8), cmap='hot')
plt.title(f'Разница (средняя по каналам)')
plt.colorbar(label='Величина различия')
plt.axis('off')

plt.tight_layout()
plt.show()

def calculate_psnr(original, compressed):
    """Вычисляет PSNR между исходным и сжатым изображениями"""
    # Переводим в float для вычислений
    original = original.astype(float)
    compressed = compressed.astype(float)

    # Вычисляем MSE (Mean Squared Error)
    mse = np.mean((original - compressed) ** 2)

    # Максимальное значение пикселя
    max_pixel = 255.0

    # Вычисляем PSNR
    if mse == 0:
        return float('inf')

    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

# Расчет PSNR для разных значений r
print("PSNR для разных значений r:")
for i, r in enumerate(r_values):
    psnr_value = calculate_psnr(x, compressed_images[i])
    print(f"r = {r}: PSNR = {psnr_value:.2f} dB, коэффициент сжатия = {compression_ratios[i]:.4f}")

# Визуализация зависимости качества от коэффициента сжатия
psnr_values = [calculate_psnr(x, compressed_images[i]) for i in range(len(r_values))]

plt.figure(figsize=(10, 6))
plt.plot(r_values, psnr_values, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Количество сингулярных значений (r)')
plt.ylabel('PSNR (dB)')
plt.title('Зависимость качества (PSNR) от количества сингулярных значений')
plt.grid(True, alpha=0.3)

# Сравнение коэффициента сжатия и PSNR
plt.figure(figsize=(10, 6))
plt.plot(compression_ratios, psnr_values, 'ro-', linewidth=2, markersize=8)
plt.xlabel('Коэффициент сжатия')
plt.ylabel('PSNR (dB)')
plt.title('Зависимость качества от коэффициента сжатия')
plt.grid(True, alpha=0.3)

# Аннотации для точек
for i, r in enumerate(r_values):
    plt.annotate(f'r={r}\n{compression_ratios[i]:.3%}',
                (compression_ratios[i], psnr_values[i]),
                textcoords="offset points",
                xytext=(0,10),
                ha='center')

plt.tight_layout()
plt.show()
# Сохранение сжатых изображений на диск
for i, r in enumerate(r_values):
    compressed_img = Image.fromarray(compressed_images[i])
    filename = f'compressed_r_{r}.jpg'
    compressed_img.save(filename, quality=95)

    # Получаем размер файла
    file_size = os.path.getsize(filename)
    print(f"r = {r}: размер файла = {file_size:,} байт, "
          f"коэффициент сжатия от исходного = {file_size/original_size_bytes:.3%}")
    print("\n" + "="*60)

print("АНАЛИЗ РЕЗУЛЬТАТОВ")
print("="*60)

print(f"\nИсходное изображение:")
print(f"  - Размер: {w}x{h} пикселей")
print(f"  - Размер файла: {original_size_bytes:,} байт")
print(f"  - Общее количество пикселей: {w*h:,}")

print(f"\nАнализ сжатия SVD:")
for i, r in enumerate(r_values):
    compressed_size = int((w + h + 1) * r * 3)  # Приблизительный размер в памяти
    file_size = os.path.getsize(f'compressed_r_{r}.jpg')
    psnr_val = calculate_psnr(x, compressed_images[i])

    print(f"\nКоличество сингулярных значений: r = {r}")
    print(f"  - Теоретический коэффициент сжатия: {compression_ratios[i]:.4%}")
    print(f"  - Размер JPEG файла: {file_size:,} байт")
    print(f"  - Качество (PSNR): {psnr_val:.2f} dB")
    print(f"  - Эффективность сжатия: {file_size/original_size_bytes:.3%} от исходного")

print(f"\nВыводы:")
print("1. SVD позволяет эффективно сжимать изображения с контролируемыми потерями.")
print("2. Качество изображения быстро улучшается с увеличением количества сингулярных значений.")
print("3. Уже при r=30 получается достаточно хорошее качество при значительном сжатии.")
print("4. При r=100 визуальное качество почти неотличимо от оригинала при сжатии ~30%.")
print("5. Реальное сжатие в файлах JPEG может отличаться от теоретического из-за алгоритма JPEG.")